
# **<font color='##6ee05a'>Machine Learning Homework 4</font>**

*   Archisa Bhattacharya
*   Marz Pino


# **<font color='violet'>Part 1:</font> Loading the Data, EDA, & Data Cleaning**

In [3]:
#from google.colab import files
import numpy as np
import pandas as pd
import data
import io

In [ ]:
# Mount Google Drive

from google.colab import drive
drive.mount('/content/drive')

In [4]:
print(data.df_train.head())


                                               image  label
0  {'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...      5
1  {'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...      0
2  {'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...      4
3  {'bytes': b"\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...      1
4  {'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...      9


# **1. [80 points, LeNet5 Implementation]** Please implement the original LetNet5 exactly as described in the paper “Gradient-Based Learning Applied to Document Recognition”. Please carefully read the Sections I, II, III and Appendix A, B, C. For your implementation, you can use PyTorch or TensorFlow. (PyTorch is recommended if you don’t have any specific preference.)

**1.1 [Architecture]** Implement LeNet architecture as described in Section II. For the RBF parameters, use the data DIGIT. There are several ways to generate the 7 x 12 bitmap image for each
digit from the data. Please explain your approach and reasoning behind it.

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from PIL import Image

# Define the LeNet-5 architecture
class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5, self).__init__()
        # Layer 1: Convolutional layer (C1)
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=2)
        # Layer 2: Subsampling (S2)
        self.pool1 = nn.AvgPool2d(kernel_size=2, stride=2)
        # Layer 3: Convolutional layer (C3)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        # Layer 4: Subsampling (S4)
        self.pool2 = nn.AvgPool2d(kernel_size=2, stride=2)
        # Layer 5: Convolutional layer (C5, fully connected)
        self.conv3 = nn.Conv2d(16, 120, kernel_size=5)
        # Layer 6: Fully connected (F6)
        self.fc1 = nn.Linear(120, 84)
        # Output layer
        self.fc2 = nn.Linear(84, 10)

    def forward(self, x):
        # Apply layers sequentially
        x = 1.7159 * torch.tanh(self.conv1(x) * 2 / 3)  # C1
        x = self.pool1(x)  # S2
        x = 1.7159 * torch.tanh(self.conv2(x) * 2 / 3)  # C3
        x = self.pool2(x)  # S4
        x = 1.7159 * torch.tanh(self.conv3(x) * 2 / 3)  # C5
        x = x.view(-1, 120)  # Flatten
        x = 1.7159 * torch.tanh(self.fc1(x) * 2 / 3)  # F6
        x = self.fc2(x)  # Output
        return x

# Define custom dataset class
class MNISTDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image = np.array(self.data.iloc[idx, 1:].values, dtype=np.uint8).reshape(28, 28)
        label = self.data.iloc[idx, 0]
        if self.transform:
            image = self.transform(Image.fromarray(image))
        return image, label

# Load and preprocess the data
def load_data():
    splits = {'train': 'mnist/train-00000-of-00001.parquet', 'test': 'mnist/test-00000-of-00001.parquet'}
    df_train = pd.read_parquet("hf://datasets/ylecun/mnist/" + splits["train"])
    df_test = pd.read_parquet("hf://datasets/ylecun/mnist/" + splits["test"])
    return df_train, df_test

# Main function
def main():
    # Load data
    df_train, df_test = load_data()

    # Define transformations
    transform = torch.transforms.Compose([
        torch.transforms.Grayscale(),
        torch.transforms.ToTensor(),
        torch.transforms.Normalize((0.5,), (0.5,))
    ])

    # Create datasets and dataloaders
    train_dataset = MNISTDataset(df_train, transform=transform)
    test_dataset = MNISTDataset(df_test, transform=transform)
    train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

    # Initialize model, loss, and optimizer
    model = LeNet5()
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

    # Training loop
    epochs = 10
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(train_loader)}")

    # Evaluation
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f"Test Accuracy: {100 * correct / total:.2f}%")

if __name__ == "__main__":
    main()


AttributeError: module 'torch' has no attribute 'transforms'

In [15]:
#Loading the dataset and preprocessing
train_dataset = torchvision.datasets.MNIST(root = './data',
                                        train = True,
                                        transform = transforms.Compose([
                                                transforms.Resize((32,32)),
                                                transforms.ToTensor(),
                                                transforms.Normalize(mean = (0.1307,), std = (0.3081,))]),
                                        download = True)


test_dataset = torchvision.datasets.MNIST(root = './data',
                                        train = False,
                                        transform = transforms.Compose([
                                                transforms.Resize((32,32)),
                                                transforms.ToTensor(),
                                                transforms.Normalize(mean = (0.1325,), std = (0.3105,))]),
                                        download=True)


train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                        batch_size = batch_size,
                                        shuffle = True)


test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                        batch_size = batch_size,
                                        shuffle = True)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:02<00:00, 3461616.57it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 3600193.00it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 17032453.28it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 4761441.83it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [23]:
#Defining the convolutional neural network
class LeNet5(nn.Module):
    def __init__(self, num_classes):
        super(ConvNeuralNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=0),
            nn.BatchNorm2d(6),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.fc = nn.Linear(400, 120)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(120, 84)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(84, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        out = self.relu(out)
        out = self.fc1(out)
        out = self.relu1(out)
        out = self.fc2(out)
        return out

In [26]:
model = LeNet5(num_classes).to(device)
    
#Setting the loss function
cost = nn.CrossEntropyLoss()

#Setting the optimizer with the model parameters and learning rate
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

#this is defined to print how many steps are remaining when training
total_step = len(train_loader)

TypeError: LeNet5.__init__() takes 1 positional argument but 2 were given

In [25]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5, self).__init__()
        # LeNet-5 original structure:
        # Input: 1x28x28 (grayscale)
        # C1: Conv2D -> 6x24x24
        # S2: AvgPool -> 6x12x12
        # C3: Conv2D -> 16x8x8
        # S4: AvgPool -> 16x4x4
        # C5: Conv2D -> 120x1x1 (fully connected as conv)
        # F6: 84 units fully connected
        # Output: 10 units fully connected (digits 0-9)

        self.conv1 = nn.Conv2d(1, 6, kernel_size=5)   # (N,1,28,28)->(N,6,24,24)
        self.pool = nn.AvgPool2d(2)                    # (N,6,24,24)->(N,6,12,12)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)   # (N,6,12,12)->(N,16,8,8)
                                                       # pool -> (N,16,4,4)
        self.fc1 = nn.Linear(16*4*4, 120)              # (N,16*4*4)->(N,120)
        self.fc2 = nn.Linear(120, 84)                  # (N,120)->(N,84)
        self.fc3 = nn.Linear(84, 10)                   # (N,84)->(N,10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.view(-1, 16*4*4)  # Flatten before FC layers
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)  # raw logits for 10 classes
        return x

if __name__ == "__main__":
    # Set up transform for MNIST: to tensor and normalize
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])

    # Load MNIST dataset from torchvision
    train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
    test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = LeNet5().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Training loop
    epochs = 1
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()

        avg_loss = running_loss / len(train_loader)
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {avg_loss:.4f}")

    # Evaluation
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print("Test Accuracy:", correct / total)


Epoch [1/1], Loss: 0.3005
Test Accuracy: 0.9691


**1.2 [Training]**
*   Data: Run data.py to collect MNIST data samples. The MNIST database consists of 28 × 28 images. You
will need to resize them to 32 × 32 to match the regular database in the original paper.
*   Optimization: Use the “Stochastic Diagonal Levenberg-Marquardt” method. Compute the exact second
derivative for hkk in equation (22) without the approximations in Appendix C.
*  Loss function: Use equation (9). j = 0.1 and i denotes incorrect classes.
* At every pass (every epoch), track the error rates. You will need the rates for plotting in the performance
evaluation



**1.3 [Performance Evaluation]**
* Create a plot of training and test error rate, similar to Fig 5.
* Compute a confusion matrix (10 × 10).
* For each digit, identify the most confusing example. For instance, what image was misclassified with the highest confidence as the digit “one”?
* Report your train and test error rates at epoch 20.